### 数据解析 Xpath
- 基本步骤：
  - 指定URL
  - 发送请求
  - 获取响应数据
  - 解析数据（Xpath表达式）
  - 存储数据库

### Xpath在爬虫中的使用流程
- 创建etree对象进行指定数据的解析
  - 本地：etree = etree.parse（本地文件路径）
  - 网络：etree = etree.HTML（网络请求到的页面数据）

In [7]:
from lxml import etree

web_data = """
        <div>
            <ul>
                 <li class="item-0"><a href="link1.html">first item</a></li>
                 <li class="item-1"><a href="link2.html">second item</a></li>
                 <li class="item-inactive"><a href="link3.html">third item</a></li>
                 <li class="item-1"><a href="link4.html">fourth item</a></li>
                 <li class="item-0"><a href="link5.html">fifth item</a>
             </ul>
         </div>
        """
element = etree.HTML(web_data)
links = element.xpath('//ul/li/a/@href')  # 获取li标签下面的a标签的href #
print(links)
result = element.xpath('//ul/li/a/text()')  # 获取li标签下面的a标签的文本数据 #
print(result)

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']
['first item', 'second item', 'third item', 'fourth item', 'fifth item']


### 以豆瓣网为例爬取电影排名的详情
#### 目标和需求
- 熟悉Xpath解析数的方式
- 爬取电影的名称 评分 引言 详情页的URL 翻页爬取1-10页 保存到列表中

#### 如何实现?
- 设计技术与需要的库 Requests lxml（etree）

#### 实现步骤
- 页面分析(一般讲数据解析模块 都是静态页面)
  - 通过观察看网页源代码中是否有我们想要的数据
    - 如果有就分析这个URL
    - 如果没有再通过Ajax寻找接口   
  - 通过分析数据在网页源代码中确定目标url
    - https：//movie.douban.com/top250?start=0&filter= 第一页
    - 通过页面分析发现所有我们想要的数据都在一个div[class="info"]里面

#### 具体实现步骤
- 获取整个网页的源码(HTML)
- 将获取的数据源码转成一个element对象（XML）
- 通过Element对象实现Xpath语法 对数据进行爬取（标题 评分 引言 详情页的URL）
- 保存数据 先保存到字典中然后保存到列表中

In [16]:
import requests
from lxml import etree

def getsource(pagelink):  # 定义一个函数用来获取网页源代码 #
    # 请求头
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    }
    response = requests.get(pagelink, headers=headers)
    html = response.text
    return html
 
def geteveryitem(html):  # 定义一个函数用于解析我们的网页源代码并获取我们想要的数据 #
    element = etree.HTML(html)
    movieitemlist = element.xpath('//li//div[@class="info"]')  # 拿到[class="info"]的所有div #
    itemlist = []
    for item in movieitemlist:
        itemdict = {}
        
        title = item.xpath('./div[@class="hd"]/a/span[@class="title"]/text()')  # 标题 #
        title = "".join(title).replace("\xa0", "")
        
        othertitle = item.xpath('./div[@class="hd"]/a/span[@class="other"]/text()')[0].replace("\xa0", "") # 副标题 #
        
        grade = item.xpath('./div[@class="bd"]/div[@class="star"]/span[2]/text()')[0]  # 评分 #
        
        link = item.xpath('div[@class="hd"]/a/@href')[0]  # 详情页的url #
        
        quote = item.xpath('div[@class="bd"]/p[@class="quote"]/span/text()')  # 引言 #
       
        if quote:  # 非空处理 #
            quote = quote[0]
        else:
            quote = ""
        
        itemdict['title'] = ''.join(title + othertitle)  # 将数据存放到字典中 #
        itemdict['grade'] = grade
        itemdict['link'] = link
        itemdict['quote'] = quote
        
        itemlist.append(itemdict)
    return itemlist
 
 
if __name__ == '__main__':
    url = 'https://movie.douban.com/top250?start=0&filter='
    html = getsource(url)
    itemlist = geteveryitem(html)
    text = ''
    for item in itemlist:
       text += str(item)
    with open('./Save_files/Movie_list.txt', 'w', encoding = 'UTF-8') as fp:  # 本地化存储 #
        fp.write(text)
    print('Save Success')

Save Success
